# Neural Networks Parameter Hypertuning

For lung cancer classification, we employed a Multilayer Perceptron (MLP), a type of feedforward neural network known for its versatility in tackling classification prediction problems. MLPs are well-suited for data presented in a tabular format, such as the features in the CSV file used.

We start by importing the necessary libraries and removing the 'id' column.

In [1]:
from kfold_and_metrics import *

import tensorflow as tf
import pandas as pd
import numpy as np

tf.keras.backend.clear_session()

In [2]:
df = pd.read_csv("final.csv")
df = df.drop(columns=['id'])
df.head()

patient_id  diagnostics_Image-original_Mean  \
0  LIDC-IDRI-0001                      -826.943929   
1  LIDC-IDRI-0001                      -826.943929   
2  LIDC-IDRI-0001                      -826.943929   
3  LIDC-IDRI-0001                      -826.943929   
4  LIDC-IDRI-0002                      -826.943929   

   diagnostics_Mask-original_VoxelNum  diagnostics_Mask-original_VolumeNum  \
0                                5905                                    1   
1                                4613                                    1   
2                                4955                                    1   
3                                5498                                    1   
4                               10351                                    1   

   diagnostics_Image-interpolated_Mean  \
0                          -417.494203   
1                          -405.581777   
2                          -410.236759   
3                          -416.576321   
4                          -546.359139   

   diagnostics_Image-interpolated_Minimum  \
0                             -990.291016   
1                             -982.456726   
2                             -990.291016   
3                             -990.291016   
4                            -1007.657349   

   diagnostics_Image-interpolated_Maximum  \
0                             1038.270874   
1                              949.768005   
2                             1038.270874   
3                             1038.270874   
4                             1020.174988   

   diagnostics_Mask-interpolated_VoxelNum  \
0                                     909   
1                                     699   
2                                     772   
3                                     841   
4                                     749   

   diagnostics_Mask-interpolated_VolumeNum  \
0                                        2   
1                                        1   
2                                        1   
3                                        2   
4                                        1   

   diagnostics_Mask-interpolated_Maximum  ...  \
0                             237.087921  ...   
1                             221.953705  ...   
2                             237.087921  ...   
3                             237.087921  ...   
4                             160.687653  ...   

   diagnostics_Mask-interpolated_BoundingBox_2  \
0                                          0.0   
1                                          0.0   
2                                          0.0   
3                                          0.0   
4                                          0.0   

   diagnostics_Mask-interpolated_BoundingBox_3  \
0                                         13.0   
1                                         13.0   
2                                         13.0   
3                                         13.0   
4                                         13.0   

   diagnostics_Mask-interpolated_BoundingBox_4  \
0                                         11.0   
1                                         11.0   
2                                         11.0   
3                                         11.0   
4                                         11.0   

   diagnostics_Mask-interpolated_BoundingBox_5  \
0                                         10.0   
1                                         10.0   
2                                         10.0   
3                                         10.0   
4                                         10.0   

   diagnostics_Mask-interpolated_CenterOfMassIndex_0  \
0                                          17.041265   
1                                          17.041265   
2                                          17.041265   
3                                          17.041265   
4                                          17.041265   

   diagnostics_Mask-interpolated_C

### Parameter Hypertuning

In the realm of machine learning, it is widely acknowledged that a single hidden layer is typically sufficient for developing an effective neural network model. With this in mind, we needed to determine the number of nodes in this hidden layer, striving to strike a balance between preventing underfitting and overfitting. To achieve this, we explored various values, using a rule of thumb being to set the maximum number of nodes to two-thirds of the number of input nodes.

Additionally, we experimented with different learning rates for the Stochastic Gradient Descent (SGD) optimizer and varied activation functions for the hidden layer.

In [3]:
best_auc = 0
best = {}

input_nodes = 100
max_n_nodes = input_nodes * 2//3

for hidden_layer_act in ["softmax", "relu", "sigmoid"]:
    for n_nodes in range(0, max_n_nodes, 10):
        for l_rate in [0.001, 0.003, 0.005, 0.007, 0.01]:
            params = {'hidden_layer_nodes': n_nodes, 'hidden_layer_activation': hidden_layer_act, 'learning_rate': l_rate}
            print("Current parameter combination:")
            for parameter, value in params.items():
                print(f"\t{parameter}: {value}")
            print()

            nn_model = tf.keras.models.Sequential([
                tf.keras.layers.Input((100,), name="input"),
                tf.keras.layers.Dense(n_nodes,activation=hidden_layer_act),
                tf.keras.layers.Dense(2,activation='softmax')
            ])
            nn_model.compile(
                optimizer=tf.keras.optimizers.SGD(l_rate), 
                loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
            )

            score = k_fold_cv_keras(compiled_model=nn_model, df=df, pca_components=100)
            results = mean_std_results_k_fold_CV(score)

            auc_avg = results.iloc[2,1]
            if auc_avg > best_auc:
                best_auc = auc_avg
                best = params

Current parameter combination:
	hidden_layer_nodes: 0
	hidden_layer_activation: softmax
	learning_rate: 0.001

Current parameter combination:
	hidden_layer_nodes: 0
	hidden_layer_activation: softmax
	learning_rate: 0.003

Current parameter combination:
	hidden_layer_nodes: 0
	hidden_layer_activation: softmax
	learning_rate: 0.005

Current parameter combination:
	hidden_layer_nodes: 0
	hidden_layer_activation: softmax
	learning_rate: 0.007

Current parameter combination:
	hidden_layer_nodes: 0
	hidden_layer_activation: softmax
	learning_rate: 0.01

Current parameter combination:
	hidden_layer_nodes: 10
	hidden_layer_activation: softmax
	learning_rate: 0.001

Current parameter combination:
	hidden_layer_nodes: 10
	hidden_layer_activation: softmax
	learning_rate: 0.003

Current parameter combination:
	hidden_layer_nodes: 10
	hidden_layer_activation: softmax
	learning_rate: 0.005

Current parameter combination:
	hidden_layer_nodes: 10
	hidden_layer_activation: softmax
	learning_rate: 0.007

After conducting our hyperparameter search, the best configuration was determined to be:

In [4]:
print("Results of the grid search parameter hypertunning:")
for parameter, value in best.items():
    print(f"\t{parameter}: {value}")

Results of the grid search parameter hypertunning:
	hidden_layer_nodes: 60
	hidden_layer_activation: relu
	learning_rate: 0.01


The number of hidden layers and rule of thumb considered can be read about in the following resources:

- "Deep Learning" by Ian Goodfellow, Yoshua Bengio, and Aaron Courville, chapter 6.4.1;
- https://medium.com/geekculture/introduction-to-neural-network-2f8b8221fbd3;
- https://stats.stackexchange.com/a/180052.